In [10]:
### Imports
import os
from typing import Dict, TypedDict, List, Union, Annotated
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, BaseMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_ollama import ChatOllama
from langgraph.graph.message import add_messages 
from dotenv import load_dotenv
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

load_dotenv()


True

In [11]:
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

In [12]:
@tool
def add(a: int, b: int) -> int:
  """Add two numbers"""
  return a + b

@tool
def subtract(a: int, b: int) -> int:
  """Subtract two numbers"""
  return a - b

tools = [add, subtract]

In [13]:
llm = ChatAnthropic(model_name="claude-haiku-4-5-20251001").bind_tools(tools)

In [14]:
def llm_invoke(state: AgentState) -> AgentState:
  system_prompt = "You are my AI assistant, please respond to my queries to best of your ability."
  response = llm.invoke([system_prompt] + state["messages"])
  return {"messages": [response]}

In [15]:
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]

    if not last_message.tool_calls:
      return "end"
    else:
      return "continue"
    

In [18]:
graph = StateGraph(AgentState)
graph.add_node("agent", llm_invoke)

tool_node = ToolNode(tools=tools)
graph.add_node("tools", tool_node)

graph.set_entry_point("agent")

graph.add_conditional_edges(
    "agent",
    should_continue,
    {
      "continue": "tools",
      "end": END
    }
)

graph.add_edge("tools", "agent")
app = graph.compile()


In [23]:
def print_stream(stream):
    for chunk in stream:
        message = chunk["messages"][-1]
        if isinstance(message, tuple):
          print(message)
        else:
          message.pretty_print()

In [26]:
inputs = {"messages": [("user", "sub 33 - 9 and add 4 + 89")]}
print_stream(app.stream(inputs, stream_mode="values"))

================================ Human Message =================================

sub 33 - 9 and add 4 + 89
================================== Ai Message ==================================

[{'text': "I'll help you with those calculations!", 'type': 'text'}, {'id': 'toolu_01Mzg2VthcreFUMoC2L2ctTE', 'input': {'a': 33, 'b': 9}, 'name': 'subtract', 'type': 'tool_use'}, {'id': 'toolu_01DhkQg9evR2Lbovn3aWHZns', 'input': {'a': 4, 'b': 89}, 'name': 'add', 'type': 'tool_use'}]
Tool Calls:
  subtract (toolu_01Mzg2VthcreFUMoC2L2ctTE)
 Call ID: toolu_01Mzg2VthcreFUMoC2L2ctTE
  Args:
    a: 33
    b: 9
  add (toolu_01DhkQg9evR2Lbovn3aWHZns)
 Call ID: toolu_01DhkQg9evR2Lbovn3aWHZns
  Args:
    a: 4
    b: 89
================================= Tool Message =================================
Name: add

93
================================== Ai Message ==================================

Here are your results:

- **33 - 9 = 24**
- **4 + 89 = 93**

Is there anything else you'd like me to help you with?
